# A database for a time-domain survey
Code accompanying the report by Charlotte Brand

In [2]:
import sys # remove at end of project
import numpy as np
import sqlite3 as lite
from astropy.table import Table
from astropy.io import fits

%matplotlib inline

In [28]:
# Reading in the info table
infofiledir = 'Tables/file_info_for_problem.csv'
#infotab = np.loadtxt(infofiledir, delimiter=',',dtype='string')
infotab = Table.read(infofiledir, format='csv')

# The filenames in "file_info_for_problem.csv" have a different format than the 
# filenames of the test data we were given. In order to use this test data we need
# to know the 'real' filenames. Below, a row with the filenames is added (filename_new)
# to the information table so that these filenames can be used to extract data from the 
# test files. 
Ks_counter = 0
newnames = []
for i, name in enumerate(infotab['Filename']):
    field = infotab['FieldID'][i]
    if i > 0 and field != infotab['FieldID'][i-1]:
        Ks_counter = 0
    band = name[:2].replace('-','') 
    newname = 'Field-'+str(field)+'-'+band
    if 'Ks' in band: 
        Ks_counter += 1
        newname = newname + '-E' + format(Ks_counter, "03")
    newname = newname+'.fits'
    newnames.append(newname)
infotab['Filename_new'] = newnames

print infotab

 ID FieldID ... Exptime     Filename_new    
--- ------- ... ------- --------------------
  1       1 ...    40.0       Field-1-Z.fits
  2       1 ...    48.0       Field-1-J.fits
  3       1 ...    16.0       Field-1-H.fits
  4       1 ...    16.0 Field-1-Ks-E001.fits
  5       1 ...    16.0 Field-1-Ks-E002.fits
  6       1 ...    16.0 Field-1-Ks-E003.fits
  7       1 ...    40.0       Field-1-Y.fits
  8       2 ...    40.0       Field-2-Z.fits
  9       2 ...    48.0       Field-2-J.fits
 10       2 ...    16.0       Field-2-H.fits
 11       2 ...    16.0 Field-2-Ks-E001.fits
 12       2 ...    40.0       Field-2-Y.fits
 13       3 ...    40.0       Field-3-Z.fits
 14       3 ...    48.0       Field-3-J.fits
 15       3 ...    16.0       Field-3-H.fits
 16       3 ...    16.0 Field-3-Ks-E001.fits
 17       3 ...    16.0 Field-3-Ks-E002.fits
 18       3 ...    40.0       Field-3-Y.fits


In [29]:
# Function used to create the database:
# (Only in a function so I don't accidentally re-run the code
# because I don't want to recreate the entire database by accident, 
# even though it's not that big)

def createdatabase():

    # Connect to database
    con = lite.connect('Database_Q1_cbrand.db')

    # Creating and filling the tables in the database:

    with con: 

        # ---------------- Creation of 7 tables below -------------------:

        # 1) The table that contains the observational info (ObsInfo)
        # ObsInfo Schema:
        ObsInfo_schema = """
                            CREATE TABLE ObsInfo (FieldID INT, Filter varchar(2), MJD DOUBLE, 
                            Filename varchar(40), 
                            UNIQUE(FieldID, Filter, MJD),
                            PRIMARY KEY(FieldID, Filter, MJD))
                         """
        # Drop if the table already exists so that changes can be made
        con.execute('DROP TABLE IF EXISTS ObsInfo')
        # Create the ObsInfo table:
        con.execute(ObsInfo_schema)

        # 2) The 5 tables that have the object data in the Z, Y, J, H and Ks filter (Data{Filter})
        tablenames = ['DataZ', 'DataY', 'DataJ', 'DataH', 'DataKs']

        for name in tablenames:
            # Data{Filter} Schema
            Data_schema = """
                            CREATE TABLE {} (StarID INT, MJD DOUBLE, 
                            Ra DOUBLE, Dec DOUBLE, X DOUBLE, Y DOUBLE, 
                            Flux1 DOUBLE, dFlux1 DOUBLE, Flux2 DOUBLE, dFlux2 DOUBLE,
                            Flux3 DOUBLE, dFlux3 DOUBLE, Class INT,
                            Mag1 DOUBLE, dMag1 DOUBLE, Mag2 DOUBLE, dMag2 DOUBLE, 
                            Mag3 DOUBLE, dMag3 DOUBLE,
                            UNIQUE(StarID, MJD),
                            PRIMARY KEY(StarID, MJD), 
                            FOREIGN KEY(StarID) REFERENCES StarFields(StarID))
                         """.format(name)
            # Drop if the table already exists so that changes can be made
            con.execute('DROP TABLE IF EXISTS {}'.format(name))
            con.execute(Data_schema)

        # 3) The table that contains info on which star is in which field (StarFields)
        # StarFields Schema:
        StarFields_schema = "CREATE TABLE StarFields (StarID INT, FieldID INT, UNIQUE(StarID), PRIMARY KEY(StarID))"
        con.execute('DROP TABLE IF EXISTS StarFields')
        con.execute(StarFields_schema)

        # ---------------- Filling of 7 tables below -------------------:

        # 1) Filling the table that contains the observational info (ObsInfo)
        # This only uses information from the file  "file_info_for_problem.csv"

        # Stack all data that we need to fill the table
        alldata_ObsInfo = np.column_stack((infotab['FieldID'], infotab['Filter'], 
                                          infotab['MJD'], infotab['Filename']))
        # Command to fill the table:
        FillObsInfo = "INSERT INTO ObsInfo VALUES (?, ?, ?, ?)"
        # Execute command
        con.executemany(FillObsInfo, alldata_ObsInfo)

        # 2) Filling the 5 tables that have the object data in the Z, Y, J, H and Ks filter (Data{Filter})
        # Only MJD is taken from the infotab table, the rest is image star data
        # MJD (the time) is taken from the information table "file_info_for_problem.csv", 
        # The rest of the data (object data) is taken from the fits files.

        # To save the unique StarIDs and FieldIDs to fill StarFields (see (3) below), create a dict
        fieldsdict = {}

        # Loop through all the available test data:
        for i, filename in enumerate(infotab['Filename_new']):

            # Open catalogue corresponding to data file 'filename':
            star_data = fits.open('Tables/'+filename)[1].data

            # Create a list of MJDs for the data file (MJD is the same for each star 
            # in one catalogue)
            MJD_list = [infotab['MJD'][i]]*len(star_data['StarID'])

            # Name of the table to fill (depends on the filter of the observation!)
            tablename = 'Data'+infotab['Filter'][i]

            # Combine all the data from the fits file that we need to put in the table
            alldata = np.column_stack((star_data['StarID'], MJD_list,
                                        star_data['Ra'], star_data['Dec'], 
                                        star_data['X'], star_data['Y'], 
                                        star_data['Flux1'], star_data['dFlux1'], 
                                        star_data['Flux2'], star_data['dFlux2'],
                                        star_data['Flux3'], star_data['dFlux3'], 
                                        star_data['Class'], 
                                        star_data['Mag1'], star_data['dMag1'], 
                                        star_data['Mag2'], star_data['dMag2'], 
                                        star_data['Mag3'], star_data['dMag3'] ))

            # Command to fill the Data table:
            FillData = """
                        INSERT INTO {0} VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                               ?, ?, ?, ?, ?, ?, ?, ?, ?)
                       """.format(tablename)

            # Execute command
            con.executemany(FillData, alldata)

            # To save the unique StarIDs and FieldIDs, put the StarIDs and FieldIDs in a dict 
            # in each loop (this will result in unique StarIDs, as they are the keys in the dict)
            fieldID_list = [infotab['FieldID'][i]]*len(star_data['StarID'])
            for key in star_data['StarID']:
                fieldsdict[key] = infotab['FieldID'][i]

        # 3) Filling the table StarFields with StarIDs and FieldIDs
        # The dictionary of unique StarIDs and corresponding FieldIDs was already filled
        # in the loop above, as we had to loop through all the files to fill the Data tables
        # anyway, otherwise we'd need to loop throught all the files again. 
        FillStarFields = "INSERT INTO StarFields VALUES(?, ?)"
        con.executemany(FillStarFields, np.column_stack((fieldsdict.keys(), fieldsdict.values())).astype(int)) 

        # TEST, REMOVE IT!!
        #test = con.execute('SELECT * from StarFields ORDER BY StarID')
        #for row in test:
            #print row

        #data = np.array(test.fetchall())

        #print data.shape
        #print len(data[:,0])
        #print len(np.unique(data[:,0]))
    con.close()

In [30]:
createdatabase()

In [31]:
# ### Filling the data tables (columns are StarID, MJD, Flux1, dFlux1, Class)
# # Only MJD is taken from the infotab table, the rest is image star data

# #fieldstotal = np.array(0,dtype=int)
# #print fieldstotal

# fieldsdict = {}

# for i, filename in enumerate(infotab['Filename_new']):
#     star_data = fits.open('Tables/'+filename+'.fits')[1].data
#     #print star_data['StarID']
#     MJD_list = [infotab['MJD'][i]]*len(star_data['StarID'])
#     fieldID_list = [infotab['FieldID'][i]]*len(star_data['StarID'])
#     tablename = 'data_'+infotab['Filter'][i]
#     print star_data['StarID']
#     for key in star_data['StarID']:
#         fieldsdict[key] = infotab['FieldID'][i]
#     fulldata = np.column_stack((star_data['StarID'], MJD_list,
#                               star_data['Flux1'], star_data['dFlux1'], 
#                               star_data['Class']))
#     #command =   """INSERT INTO {0} VALUES({1},{2},{3},{4},{5})
#     #                """.format(tablename, star_data['StarID'], MJD_list,
#     #                          star_data['Flux1'], star_data['dFlux1'], 
#     #                          star_data['Class'])
#     command =   """INSERT INTO {0} VALUES(?, ?, ?, ?, ?)
#                     """.format(tablename)
#     #print commandmany
#     con.executemany(command, fulldata)
    
# # Fill fields: StarID INT, FieldID INT
# command = "INSERT INTO fields VALUES(?, ?)"
# con.executemany(command, np.column_stack((fieldsdict.keys(), fieldsdict.values())).astype(int)) #works
# # can sort by starID in query ^ if you want
# test = con.execute('SELECT * from fields')
# #for row in test:
#     #print row
    
# data = np.array(test.fetchall())

# print data.shape
# print len(data[:,0])
# print len(np.unique(data[:,0]))


In [32]:
# # Imageinfo fill: FieldID INT, Filter varchar(2), MJD FLOAT, Filename varchar(40)
# fulldata_imginfo = np.column_stack((infotab['FieldID'], infotab['Filter'], 
#                                   infotab['MJD'], infotab['Filename']))
# command = "INSERT INTO imageinfo VALUES (?, ?, ?, ?)"
# con.executemany(command, fulldata_imginfo)

In [48]:
# # Close the connection 
# con.close()
# Open connec again:
con = lite.connect('Database_Q1_cbrand.db')

# Some testing:

res = con.execute("SELECT name FROM sqlite_master WHERE type='table'")
print ' Tables in the database:'
for row in res: 
    print row[0]
    
res = con.execute("SELECT * from ObsInfo")
print '\n Columns in obsinfo'
for row in res.description:
    print row[0]
    
res = con.execute("SELECT * from DataZ")
print '\n Columns in DataZ'
for row in res.description:
    print row[0]

res = con.execute("SELECT * from StarFields")
print '\n Columns in StarFields'
for row in res.description:
    print row[0]

print '\n Values in Dataz'
res = con.execute("SELECT * FROM DataZ")
for i, row in enumerate(res):
    print i, row
    if i == 5: break

print '\n Values in ObsInfo'
res = con.execute("SELECT * FROM ObsInfo")
for i, row in enumerate(res):
    print i, row
    if i == 5: break
        
print '\n Values in StarFields'
res = con.execute("SELECT * FROM StarFields")
for i, row in enumerate(res):
    print i, row
    if i == 5: break
"""        
        
# Looks like the database is good now!



'\nres = con.execute("SELECT name FROM sqlite_master WHERE type=\'table\'")\nprint \' Tables in the database:\'\nfor row in res: \n    print row[0]\n    \nres = con.execute("SELECT * from ObsInfo")\nprint \'\n Columns in obsinfo\'\nfor row in res.description:\n    print row[0]\n    \nres = con.execute("SELECT * from DataZ")\nprint \'\n Columns in DataZ\'\nfor row in res.description:\n    print row[0]\n\nres = con.execute("SELECT * from StarFields")\nprint \'\n Columns in StarFields\'\nfor row in res.description:\n    print row[0]\n\nprint \'\n Values in Dataz\'\nres = con.execute("SELECT * FROM DataZ")\nfor i, row in enumerate(res):\n    print i, row\n    if i == 5: break\n\nprint \'\n Values in ObsInfo\'\nres = con.execute("SELECT * FROM ObsInfo")\nfor i, row in enumerate(res):\n    print i, row\n    if i == 5: break\n        \nprint \'\n Values in StarFields\'\nres = con.execute("SELECT * FROM StarFields")\nfor i, row in enumerate(res):\n    print i, row\n    if i == 5: break\n'

In [12]:
#R1: 
r10 = "SELECT * from imageinfo WHERE MJD >= 56800 AND MJD <= 57300 ORDER BY MJD"
r10ex = con.execute(r10)
# NOTE: sort data info by measurement time?
for row in r10ex:
    # calc number of stars with S/N > 5 for each image
    # time interval already satisfied but do again
    # Check S/N calc
    r11 = """   SELECT COUNT(*) FROM data_{0} AS dat, fields
                WHERE dat.StarID == fields.StarID
                AND dat.Class == -1
                AND fields.FieldID == {1}
                AND dat.MJD == {2}
                AND (dat.Flux1/dat.dFlux1) > 5
            """.format(row[1], row[0], row[2])
#     r11 = """   SELECT COUNT(*) FROM data_{0} AS dat, fields
#                 WHERE dat.StarID == fields.StarID
#                 AND dat.Class == -1
#                 AND dat.MJD == {1}
#                 AND (dat.Flux1/dat.dFlux1) > 5
#             """.format(row[1], row[2])
    r11ex = con.execute(r11)
    r11res = r11ex.fetchall()
    print 'Filename: ', row[3], ', MJD: ', row[2], '#stars:', r11res[0][0]
    #print 'Filename: ', row[3], ', MJD: ', row[2], len(r11res) #same as above

    


Filename:  Ks-ADP.2016-05-25T15:33:43.377.fits , MJD:  56829.0390512 #stars: 7888
Filename:  H-ADP.2017-01-18T11:58:35.780.fits , MJD:  57257.044108 #stars: 7982
Filename:  J-ADP.2017-01-18T11:58:35.781.fits , MJD:  57257.0504323 #stars: 7022
Filename:  H-ADP.2017-01-18T11:58:35.780b.fits , MJD:  57258.044108 #stars: 7725
Filename:  H-ADP.2017-01-18T11:58:35.780c.fits , MJD:  57258.044108 #stars: 8022
Filename:  J-ADP.2017-01-18T11:58:35.781b.fits , MJD:  57258.0504323 #stars: 7354
Filename:  J-ADP.2017-01-18T11:58:35.781c.fits , MJD:  57258.0504323 #stars: 7248
Filename:  Y-ADP.2017-01-18T11:58:36.901.fits , MJD:  57267.1596647 #stars: 6806
Filename:  Z-ADP.2017-01-18T11:58:36.905.fits , MJD:  57267.1671072 #stars: 6477
Filename:  Y-ADP.2017-01-18T11:58:36.901b.fits , MJD:  57268.1596647 #stars: 7215
Filename:  Y-ADP.2017-01-18T11:58:36.901c.fits , MJD:  57268.1596647 #stars: 7186
Filename:  Z-ADP.2017-01-18T11:58:36.905b.fits , MJD:  57268.1671072 #stars: 6929
Filename:  Z-ADP.2017-0

In [39]:
#R2:
r20 =   """ SELECT * from data_J, data_H
            WHERE data_J.StarID == data_H.StarID
            AND data_J.Mag1 - data_H.Mag2 > 1.5
        """
r20ex = con.execute(r20)
for row in r20ex:
    print row

OperationalError: no such column: data_J.Mag1

In [50]:
#R3 
r3 = """
        SELECT * FROM data_Ks
        WHERE ABS( Flux1 - (SELECT AVG(Flux1) FROM data_Ks
        GROUP BY data_Ks.StarID) ) > ABS(20*dFlux1)
     """
r3 = """
        SELECT * FROM data_Ks
        WHERE ABS( Flux1 - (SELECT AVG(Flux1) FROM data_Ks
        GROUP BY data_Ks.StarID) ) > ABS(20*dFlux1)
        ORDER BY StarID
     """
r3res = con.execute(r3)
for row in r3res:
    print row
    
# Note: classification differs for same StarID (can)
# Minstens 1 van de observaties verschillend van de mean? 
# Of allemaal? -> Nee: smooth data zou altijd een deel gelijk moeten zijn aan de mean
# Lijkt te werken, check (SQL?) hoeveel van de sterren dit zijn!

(0, 56561.0020158, 4714.59912109375, 144.59144592285156, -1)
(0, 56829.0390512, 15381.96875, 179.19769287109375, -1)
(1, 56561.0020158, 5876.4208984375, 146.33033752441406, 1)
(1, 56829.0390512, 13778.82421875, 178.47972106933594, 1)
(2, 56561.0020158, 5616.17138671875, 145.0798797607422, -1)
(2, 56829.0390512, 13811.03125, 178.43405151367188, -1)
(3, 56561.0020158, 1471.169677734375, 172.43528747558594, -1)
(3, 56788.346937, 1126.1964111328125, 175.40489196777344, -1)
(3, 56829.0390512, 2551.41015625, 175.12017822265625, 1)
(4, 56829.0390512, 17139.7734375, 182.5265655517578, -1)
(5, 56829.0390512, 21366.841796875, 182.21336364746094, -1)
(6, 56829.0390512, 19162.451171875, 181.07305908203125, -1)
(7, 56561.0020158, 4168.158203125, 144.34414672851562, -2)
(7, 56788.346937, 4280.4443359375, 181.27566528320312, -1)
(8, 56788.346937, 13750.5244140625, 180.8615264892578, -1)
(8, 56829.0390512, 33430.48046875, 190.6881103515625, -1)
(9, 56829.0390512, 14549.078125, 178.80511474609375, -1)


(889, 56561.0020158, 2545.84765625, 158.94427490234375, -1)
(889, 56788.346937, 2499.94384765625, 186.07467651367188, -1)
(890, 56561.0020158, 2411.435302734375, 181.00100708007812, 1)
(890, 56788.346937, 2169.859375, 184.42298889160156, -2)
(890, 56829.0390512, 4956.12060546875, 193.2369384765625, -1)
(891, 56561.0020158, 2669.06787109375, 202.3297576904297, -1)
(891, 56788.346937, 2328.841064453125, 207.2045135498047, -1)
(891, 56829.0390512, 5222.259765625, 189.070068359375, -1)
(892, 56561.0020158, 85196.828125, 231.00274658203125, -1)
(892, 56788.346937, 70725.2890625, 216.00466918945312, -1)
(892, 56829.0390512, 109239.234375, 238.95684814453125, -1)
(893, 56561.0020158, 3324.159423828125, 159.3640899658203, -1)
(893, 56788.346937, 3063.859130859375, 186.37437438964844, -1)
(894, 56561.0020158, 2100.201171875, 187.36309814453125, -1)
(894, 56788.346937, 1805.473876953125, 179.8497314453125, -1)
(894, 56829.0390512, 4256.86083984375, 176.8536834716797, -1)
(895, 56561.0020158, 253

(2200, 56561.0020158, 4889.34375, 170.9247589111328, 1)
(2200, 56788.346937, 4046.4921875, 180.51893615722656, 1)
(2201, 56561.0020158, 4403.36181640625, 170.9490203857422, -1)
(2201, 56788.346937, 4084.142333984375, 180.5318145751953, -1)
(2202, 56829.0390512, 17763.837890625, 191.15074157714844, -1)
(2203, 56561.0020158, 58883.5859375, 186.46475219726562, -1)
(2203, 56788.346937, 55703.125, 213.52821350097656, -1)
(2203, 56829.0390512, 132238.234375, 237.6920166015625, -1)
(2204, 56561.0020158, 5288.45361328125, 170.16757202148438, -1)
(2204, 56788.346937, 4246.24072265625, 190.18112182617188, -1)
(2205, 56561.0020158, 1711.714111328125, 160.9267120361328, -1)
(2205, 56788.346937, 1381.100341796875, 193.15216064453125, 1)
(2205, 56829.0390512, 3519.807861328125, 177.4940185546875, 1)
(2206, 56561.0020158, 63116.8515625, 216.1626434326172, -1)
(2206, 56788.346937, 55302.9453125, 208.4547576904297, -1)
(2206, 56829.0390512, 106702.34375, 222.06735229492188, -1)
(2207, 56561.0020158, 49

(3550, 56829.0390512, 29495.275390625, 188.77101135253906, -1)
(3551, 56829.0390512, 13443.4453125, 188.96966552734375, -2)
(3552, 56561.0020158, 4414.1826171875, 188.62637329101562, -2)
(3552, 56788.346937, 3839.16845703125, 180.79092407226562, -2)
(3553, 56561.0020158, 1593.603515625, 189.9234161376953, -1)
(3553, 56788.346937, 1256.5230712890625, 182.04273986816406, -1)
(3553, 56829.0390512, 3547.111572265625, 190.7922821044922, -1)
(3554, 56561.0020158, 1732.2987060546875, 170.1215362548828, 1)
(3554, 56788.346937, 1601.7890625, 191.64779663085938, 1)
(3554, 56829.0390512, 2691.850341796875, 191.0888671875, 1)
(3555, 56829.0390512, 16940.5703125, 198.2428436279297, -1)
(3556, 56561.0020158, 27508.548828125, 183.61717224121094, -1)
(3556, 56788.346937, 27344.498046875, 204.87649536132812, -1)
(3556, 56829.0390512, 61099.1328125, 219.0060272216797, -1)
(3557, 56561.0020158, 3383.19482421875, 171.03033447265625, 1)
(3557, 56788.346937, 3398.949951171875, 192.61680603027344, 1)
(3558, 

(4867, 56788.346937, 2931.65087890625, 180.4515838623047, -1)
(4868, 56561.0020158, 107507.8984375, 237.87782287597656, -1)
(4868, 56788.346937, 97200.046875, 224.90072631835938, -1)
(4868, 56829.0390512, 156669.71875, 242.53817749023438, -1)
(4869, 56561.0020158, 25213.3203125, 199.56723022460938, -1)
(4869, 56788.346937, 23233.650390625, 190.8472442626953, -1)
(4869, 56829.0390512, 50101.2109375, 198.927490234375, -1)
(4870, 56561.0020158, 80422.2109375, 207.76182556152344, -1)
(4870, 56788.346937, 68188.5859375, 211.27638244628906, -1)
(4870, 56829.0390512, 128042.5390625, 230.27212524414062, -1)
(4871, 56561.0020158, 5463.18115234375, 177.97769165039062, 1)
(4871, 56788.346937, 4903.4501953125, 183.28945922851562, -1)
(4872, 56561.0020158, 3336.754150390625, 175.85919189453125, 1)
(4872, 56788.346937, 2934.771240234375, 182.21971130371094, 1)
(4873, 56561.0020158, 3497.338623046875, 175.0196533203125, 1)
(4873, 56788.346937, 3089.024169921875, 182.3689422607422, 1)
(4874, 56561.002

...

(301152, 56789.346937, 205291.875, 280.671142578125, -1)
(301153, 56562.0020158, 27648.07421875, 203.2866973876953, -1)
(301153, 56789.346937, 23953.689453125, 203.73995971679688, 1)
(301154, 56562.0020158, 181143.15625, 270.33837890625, -1)
(301154, 56789.346937, 208809.15625, 281.4825134277344, -1)
(301155, 56562.0020158, 24563.69140625, 201.3414764404297, 1)
(301155, 56789.346937, 22291.1953125, 202.98866271972656, 1)
(301157, 56562.0020158, 2784.1845703125, 168.03810119628906, 1)
(301157, 56789.346937, 2619.781005859375, 185.42405700683594, 1)
(301158, 56562.0020158, 19858.572265625, 162.3603973388672, -1)
(301158, 56789.346937, 18094.833984375, 209.30975341796875, -1)
(301159, 56562.0020158, 48833.14453125, 176.56126403808594, -1)
(301159, 56789.346937, 46241.77734375, 223.141357421875, -1)
(301160, 56562.0020158, 2354.259765625, 144.48526000976562, -1)
(301160, 56789.346937, 2240.060546875, 192.75909423828125, -1)
(301161, 56562.0020158, 1704.6341552734375, 144.13113403320312, 1)

(303181, 56789.346937, 16617.9375, 199.0357208251953, 1)
(303183, 56562.0020158, 107579.453125, 198.65830993652344, -1)
(303183, 56789.346937, 170071.5, 269.3631896972656, -1)
(303184, 56562.0020158, 4781.013671875, 150.7993621826172, 1)
(303184, 56789.346937, 4344.7080078125, 195.83274841308594, 1)
(303185, 56562.0020158, 26696.849609375, 162.09487915039062, -1)
(303185, 56789.346937, 24702.3125, 206.16627502441406, -1)
(303186, 56562.0020158, 70176.953125, 182.67176818847656, -1)
(303186, 56789.346937, 65003.9296875, 225.46376037597656, -1)
(303188, 56562.0020158, 137684.75, 214.4115447998047, -1)
(303188, 56789.346937, 150327.90625, 268.30615234375, -1)
(303189, 56562.0020158, 39060.19921875, 171.9154052734375, -1)
(303189, 56789.346937, 37027.140625, 218.70899963378906, -1)
(303190, 56562.0020158, 13675.748046875, 159.15188598632812, -1)
(303191, 56562.0020158, 48944.6328125, 176.62953186035156, -1)
(303191, 56789.346937, 45265.55859375, 222.658203125, -1)
(303192, 56562.0020158, 2

(305168, 56562.0020158, 3217.3095703125, 190.23794555664062, -1)
(305168, 56789.346937, 2632.671630859375, 192.6970977783203, -1)
(305169, 56562.0020158, 3854.886962890625, 190.9468536376953, -1)
(305169, 56789.346937, 3388.35009765625, 193.13316345214844, -1)
(305170, 56562.0020158, 4776.18408203125, 172.8044891357422, -1)
(305170, 56789.346937, 3910.9404296875, 187.23995971679688, -1)
(305173, 56562.0020158, 1134.2705078125, 143.82032775878906, 1)
(305173, 56789.346937, 1052.4959716796875, 192.12191772460938, 1)
(305175, 56562.0020158, 1389.9111328125, 143.96005249023438, -1)
(305175, 56789.346937, 1144.124267578125, 192.1750946044922, 1)
(305176, 56562.0020158, 3071.670166015625, 144.87767028808594, 1)
(305176, 56789.346937, 2905.821533203125, 193.116943359375, -1)
(305177, 56562.0020158, 28951.001953125, 158.28395080566406, -1)
(305177, 56789.346937, 26736.70703125, 205.34620666503906, -1)
(305178, 56789.346937, 5094.58251953125, 187.6663360595703, -1)
(305179, 56562.0020158, 2060.

(307219, 56562.0020158, 2263.9765625, 189.89837646484375, -1)
(307219, 56789.346937, 1932.6595458984375, 192.36383056640625, -1)
(307220, 56562.0020158, 39390.16796875, 208.7859344482422, -1)
(307220, 56789.346937, 35132.50390625, 209.28585815429688, -1)
(307222, 56562.0020158, 15108.455078125, 196.593505859375, -1)
(307222, 56789.346937, 13747.3759765625, 198.5768280029297, -1)
(307224, 56562.0020158, 3145.87939453125, 172.31053161621094, -2)
(307224, 56789.346937, 2695.95068359375, 186.5558319091797, -2)
(307225, 56562.0020158, 1075.2864990234375, 169.67257690429688, -1)
(307225, 56789.346937, 811.075927734375, 180.98260498046875, 1)
(307226, 56562.0020158, 4561.81005859375, 173.26483154296875, -1)
(307226, 56789.346937, 4044.681884765625, 188.48377990722656, -1)
(307227, 56562.0020158, 12644.029296875, 154.8119659423828, -1)
(307228, 56562.0020158, 15824.1240234375, 171.77145385742188, 1)
(307228, 56789.346937, 14584.650390625, 188.03883361816406, -1)
(307229, 56562.0020158, 2146.68

(309315, 56562.0020158, 3211.98193359375, 187.5975341796875, -1)
(309315, 56789.346937, 2741.80078125, 189.9264373779297, -1)
(309316, 56562.0020158, 1300.57861328125, 163.81680297851562, -1)
(309316, 56789.346937, 1043.330810546875, 186.55897521972656, -1)
(309317, 56562.0020158, 1639.4315185546875, 177.4445037841797, -1)
(309317, 56789.346937, 1457.4591064453125, 187.94515991210938, -1)
(309318, 56562.0020158, 2349.69580078125, 172.071533203125, -1)
(309318, 56789.346937, 2102.6884765625, 188.10055541992188, -1)
(309320, 56562.0020158, 23087.56640625, 182.91075134277344, -1)
(309320, 56789.346937, 20727.998046875, 197.44119262695312, -1)
(309321, 56562.0020158, 4811.55224609375, 173.5208282470703, -1)
(309321, 56789.346937, 4482.0078125, 188.83767700195312, -2)
(309323, 56562.0020158, 24286.498046875, 188.4000244140625, -1)
(309323, 56789.346937, 21117.017578125, 201.19471740722656, -1)
(309324, 56562.0020158, 1438.165283203125, 148.1363983154297, -1)
(309324, 56789.346937, 990.99786